In [1]:
import numpy as np
import cv2
import pandas as pd

In [38]:
data = pd.read_fwf('data/Gliding.txt', header=None)
data.columns = ['latitude', 'longitude']

In [60]:
cap = cv2.VideoCapture('video_file/Gliding.mp4')
index = 0
while(cap.isOpened()):
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    x, y, z = conversion(960, data.iloc[index].latitude, data.iloc[index].longitude)

    # cv2.circle(frame, (np.float32(1920 + data.iloc[index].latitude), np.float32(960 + data.iloc[index].longitude)), 10, (0, 255, 0), 10)
    cv2.circle(frame, (np.float32(1920 + x), np.float32(960 + y)), 10, (0, 255, 0), 10)

    cv2.namedWindow('frame', 0)
    cv2.resizeWindow('frame', 1920,1080) 
    cv2.imshow('frame',frame)

    index += 1
    # print(index)
    if (cv2.waitKey(1) & 0xFF == ord('q')) or index>300:
        break

cap.release()
cv2.destroyAllWindows()

In [26]:
cap = cv2.VideoCapture('video_file/Gliding.mp4')

In [27]:
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [28]:
# ret, frame = cap.read()

In [29]:
print(length)

625


In [30]:
len(data)

498

In [47]:
def conversion(R, lat, lon):
    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    z = R * np.sin(lat)
    return(x, y, z)

In [54]:
conversion(960, data.iloc[100].latitude, data.iloc[100].longitude)

(780.9858557631538, 379.78733440108095, -409.17315860942756)